In [ ]:
!pip install tensorflow
!pip install tensorflow-gpu
!pip install opencv-python
!pip install matplotlib


import tensorflow as tf
import numpy as np
import imghdr
import json
import cv2
import os

from datetime import datetime
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [ ]:
#avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

#Set training data directory    
data_dir = 'training_data'

#define required image extensions
image_exts = ['jpeg','jpg','bmp','png']

#remove unwanted images
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)
            
#read and create a dataset of images with labels            
data = tf.keras.utils.image_dataset_from_directory('training_data')


#access images as numpy array
data_iterator = data.as_numpy_iterator()
batch = data_iterator.next()


# Generate a unique folder name using the current time and date to save model files
now = datetime.now().strftime("%d.%m.%Y %H%M%S")
folder_name = f"{now}-train_results"
folder_path = os.path.join(folder_name)
os.makedirs(folder_path, exist_ok=True)

#create batch of image data and labels
batch[0]
batch[1]
    
# Plot the batch results
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])    

# Save the plot figure to a file
fig.savefig(os.path.join(folder_path, 'batch_result.jpg'))

    

data = data.map(lambda x,y: (x/255, y))
data = tf.keras.utils.image_dataset_from_directory('training_data')

train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

model = Sequential()

model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

model.summary()







# Get current date and time
now = datetime.now()

# Save model summary to a text file
with open(f'{folder_name}/summary.txt', 'w') as f:
    f.write(f'Summary generated on {now}\n\n')
    model.summary(print_fn=lambda x: f.write(x + '\n'))


logdir = os.path.join(folder_path, 'logs')

# Check if the logs directory already exists
if not os.path.exists(logdir):
  # Create the logs directory if it does not exist
  os.makedirs(logdir)

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

hist.history


  

with open(os.path.join(folder_path, 'history.json'), 'w') as f:
    json.dump(hist.history, f)





# Plot the accuracy chart
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")

# Save the figure to a file
fig.savefig(os.path.join(folder_path, 'accuracy_chart.jpg'))







# Plot the loss chart
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")

# Save the figure to a file

fig.savefig(os.path.join(folder_path, 'loss_chart.jpg'))


pre = Precision()
re = Recall()
acc = BinaryAccuracy()

for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)
    
print(f'Precision:{pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')    




# Open the file in write mode
with open(os.path.join(folder_path, 'results.txt'), 'w') as f:
    # Write the results to the file
    f.write(f'Precision:{pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')



    

model_name_input = input('Enter the model name to save: ')
model_name = f'{model_name_input}_model.h5'
model.save(os.path.join(folder_path, model_name))

 
